# Building a Basic RAG Agent with GoodMem

## Overview

This tutorial will guide you through building a complete **Retrieval-Augmented Generation (RAG)** system using GoodMem's vector memory capabilities. By the end of this guide, you'll have a functional Q&A system that can:

- 🔍 **Semantically search** through your documents
- 📝 **Generate contextual answers** using retrieved information 
- 🏗️ **Scale to handle** large document collections

### What is RAG?

RAG combines the power of **retrieval** (finding relevant information) with **generation** (creating natural language responses). This approach allows AI systems to provide accurate, context-aware answers by:

1. **Retrieving** relevant documents from a knowledge base
2. **Augmenting** the query with this context
3. **Generating** a comprehensive answer using both the query and retrieved information

### Why GoodMem for RAG?

GoodMem provides enterprise-grade vector storage with:
- **Multiple embedder support** for optimal retrieval accuracy
- **Streaming APIs** for real-time responses
- **Advanced post-processing** with reranking and summarization
- **Scalable architecture** for production workloads


## Prerequisites

Before starting, ensure you have:

- ✅ **GoodMem server running** (install with: `curl -s https://get.goodmem.ai | bash`)
- ✅ **Node.js 14+** installed
- ✅ **npm or yarn** for package management
- ✅ **API key** for your GoodMem instance
- ✅ **OpenAI API key** (optional, for the complete RAG demo)

## Installation & Setup

First, let's install the required packages:

In [ ]:
#!javascript
// Install the GoodMem client library
// Run in your terminal: npm install @pairsystems/goodmem-client --save

console.log('📦 To install dependencies, run:');
console.log('   npm install @pairsystems/goodmem-client');
console.log('\n💡 Make sure Node.js 14+ is installed');

## Authentication & Configuration

### Why This Matters

GoodMem uses API key authentication to secure your vector memory data. Proper configuration ensures:
- **Secure access** to your GoodMem instance
- **Isolated environments** (development, staging, production)
- **Usage tracking** and access control per API key

### What We'll Do

1. Configure the GoodMem host URL (where your server is running)
2. Set up API key authentication
3. Verify the configuration is correct

### Configuration Options

- **Local development**: `http://localhost:8080` (default)
- **Remote/production**: Your deployed GoodMem URL
- **Environment variables**: Best practice for managing credentials

Let's configure our GoodMem client and test the connection:

In [ ]:
const GoodMemClient = require('@pairsystems/goodmem-client');
const fs = require('fs').promises;
const path = require('path');

// Configuration - Update these values for your setup
const GOODMEM_HOST = process.env.GOODMEM_HOST || 'http://localhost:8080';
const GOODMEM_API_KEY = process.env.GOODMEM_API_KEY || 'your-api-key-here';

console.log('GoodMem Host:', GOODMEM_HOST);
console.log('API Key configured:', GOODMEM_API_KEY !== 'your-api-key-here' ? 'Yes' : 'No - Please update');

// Create and configure API client
const apiClient = new GoodMemClient.ApiClient();
apiClient.basePath = GOODMEM_HOST;
apiClient.defaultHeaders = {
    'X-API-Key': GOODMEM_API_KEY,
    'Content-Type': 'application/json',
    'Accept': 'application/json'
};

// Create API instances
const spacesApi = new GoodMemClient.SpacesApi(apiClient);
const memoriesApi = new GoodMemClient.MemoriesApi(apiClient);
const embeddersApi = new GoodMemClient.EmbeddersApi(apiClient);

// Create streaming client
let streamingClient = null;
try {
    streamingClient = new GoodMemClient.StreamingClient(apiClient);
    console.log('✅ StreamingClient initialized successfully');
} catch (error) {
    console.log('⚠️  StreamingClient not available:', error.message);
}

console.log('✅ GoodMem client configured successfully!');

In [ ]:
// Test connection by listing existing spaces
spacesApi.listSpaces()
    .then(response => {
        console.log('✅ Successfully connected to GoodMem!');
        const spaces = response.spaces || [];
        console.log(`   Found ${spaces.length} existing spaces`);
    })
    .catch(error => {
        console.log('❌ Error connecting to GoodMem:', error.message);
        console.log('   Please check your API key and host configuration');
        if (error.status) {
            console.log('   Response code:', error.status);
        }
    });

## Creating an Embedder

### Why Embedders Matter

An **embedder** is the foundation of semantic search. It converts text into high-dimensional vectors (embeddings) that capture meaning:

```
Text: "vacation policy" → Vector: [0.23, -0.45, 0.67, ...]  (1536 dimensions)
```

These vectors enable:
- **Semantic similarity**: Find conceptually similar content, not just keyword matches
- **Context understanding**: Capture meaning beyond exact word matches
- **Efficient retrieval**: Fast vector comparisons using specialized indexes

### The RAG Pipeline Flow

```
Documents → Embedder → Vector Storage → Semantic Search → Retrieved Context
```

### Choosing an Embedder

**OpenAI `text-embedding-3-small`** (what we'll use):
- ✅ **High quality**: Excellent for most use cases
- ✅ **Fast**: Low latency for real-time applications  
- ✅ **1536 dimensions**: Good balance of quality and storage
- ✅ **Cost-effective**: $0.02 per 1M tokens

**Other options**:
- **text-embedding-3-large**: Higher quality, 3072 dimensions, more expensive
- **Voyage AI**: Specialized for search, excellent retrieval performance
- **Cohere**: Good multilingual support
- **Local models**: HuggingFace sentence transformers for privacy/offline

### What We'll Do

1. Check if an embedder already exists
2. If not, create an OpenAI embedder with proper authentication
3. Verify the embedder is ready for use

**Note**: You'll need an OpenAI API key set in your environment variable `OPENAI_API_KEY`.

In [ ]:
// JavaScript SDK embedder creation - Coming soon


## Creating Your First Space

### What is a Space?

A **Space** in GoodMem is a logical container for organizing related memories (documents). Think of it as a database or collection where you store and retrieve semantically similar content.

Each space has:
- **Associated embedders**: Which models convert text to vectors
- **Chunking configuration**: How documents are split into searchable pieces
- **Access controls**: Public or private, with permission management
- **Metadata labels**: For organization and filtering

### Use Cases for Multiple Spaces

You might create different spaces for:
- **By domain**: Technical docs, HR policies, product specs
- **By environment**: Development, staging, production
- **By customer**: Tenant-specific data in multi-tenant apps
- **By privacy level**: Public FAQ vs. internal knowledge base

### Why Chunking Matters

Documents are too large to search efficiently as whole units. Chunking:
- **Improves relevance**: Match specific sections, not entire documents
- **Enables context**: Return focused chunks that answer specific questions  
- **Optimizes retrieval**: Process and compare smaller text segments

**Our chunking strategy**:
- **256 characters**: Short enough for focused context, long enough for meaning
- **25 character overlap**: Ensures concepts spanning chunk boundaries aren't lost
- **Hierarchical separators**: Split on paragraphs first, then sentences, then words

### What We'll Do

1. List available embedders
2. Create a space with our embedder and chunking configuration
3. Add metadata labels for organization
4. Verify the space is ready

Let's create a space for our RAG demo:

In [ ]:
// First, let's see what embedders are available
let availableEmbedders = [];
let defaultEmbedder = null;

embeddersApi.listEmbedders()
    .then(embeddersResponse => {
        availableEmbedders = embeddersResponse.embedders || [];
        
        console.log(`📋 Available Embedders (${availableEmbedders.length}):`);
        availableEmbedders.forEach((embedder, i) => {
            console.log(`   ${i + 1}. ${embedder.displayName} - ${embedder.providerType}`);
            console.log(`      Model: ${embedder.modelIdentifier || 'N/A'}`);
            console.log(`      ID: ${embedder.embedderId}`);
            console.log();
        });
        
        if (availableEmbedders.length > 0) {
            defaultEmbedder = availableEmbedders[0];
            console.log(`🎯 Using embedder: ${defaultEmbedder.displayName}`);
        } else {
            console.log('⚠️  No embedders found. You may need to configure an embedder first.');
            console.log('   Refer to the documentation: https://docs.goodmem.ai/docs/reference/cli/goodmem_embedder_create/');
        }
    })
    .catch(error => {
        console.log('❌ Error listing embedders:', error.message);
    });

In [ ]:
// Create a space for our RAG demo
const SPACE_NAME = 'RAG Demo Knowledge Base (JavaScript)';
let demoSpace = null;

// Define chunking configuration that we'll reuse throughout the tutorial
const DEMO_CHUNKING_CONFIG = {
    recursive: {
        chunkSize: 256,
        chunkOverlap: 25,
        separators: ['\n\n', '\n', '. ', ' ', ''],
        keepStrategy: 'KEEP_END',
        separatorIsRegex: false,
        lengthMeasurement: 'CHARACTER_COUNT'
    }
};

console.log('📋 Demo Chunking Configuration:');
console.log(`   Chunk Size: ${DEMO_CHUNKING_CONFIG.recursive.chunkSize} characters`);
console.log(`   Overlap: ${DEMO_CHUNKING_CONFIG.recursive.chunkOverlap} characters`);
console.log(`   Strategy: ${DEMO_CHUNKING_CONFIG.recursive.keepStrategy}`);
console.log('   💡 This chunking config will be reused for all memory creation!');
console.log();

spacesApi.listSpaces()
    .then(existingSpaces => {
        // Check if space already exists
        const spaces = existingSpaces.spaces || [];
        const existing = spaces.find(space => space.name === SPACE_NAME);
        
        if (existing) {
            console.log(`📁 Space '${SPACE_NAME}' already exists`);
            console.log(`   Space ID: ${existing.spaceId}`);
            console.log('   To remove existing space, see https://docs.goodmem.ai/docs/reference/cli/goodmem_space_delete/');
            demoSpace = existing;
            return existing;
        }
        
        // Create space if it doesn't exist
        const spaceEmbedders = [];
        if (defaultEmbedder) {
            spaceEmbedders.push({
                embedderId: defaultEmbedder.embedderId,
                defaultRetrievalWeight: 1.0
            });
        }
        
        const createRequest = {
            name: SPACE_NAME,
            labels: {
                purpose: 'rag-demo',
                environment: 'tutorial',
                'content-type': 'documentation',
                language: 'javascript'
            },
            spaceEmbedders: spaceEmbedders,
            publicRead: false,
            defaultChunkingConfig: DEMO_CHUNKING_CONFIG
        };
        
        return spacesApi.createSpace(createRequest);
    })
    .then(space => {
        if (!demoSpace) {
            demoSpace = space;
            console.log(`✅ Created space: ${space.name}`);
            console.log(`   Space ID: ${space.spaceId}`);
            console.log(`   Embedders: ${space.spaceEmbedders ? space.spaceEmbedders.length : 0}`);
            console.log(`   Labels: ${JSON.stringify(space.labels)}`);
            console.log(`   Chunking Config Saved: ${DEMO_CHUNKING_CONFIG.recursive.chunkSize} chars with ${DEMO_CHUNKING_CONFIG.recursive.chunkOverlap} overlap`);
        }
    })
    .catch(error => {
        console.log('❌ Error creating space:', error.message);
        if (error.status) {
            console.log('   Response code:', error.status);
        }
    });

In [ ]:
// Verify our space configuration
if (demoSpace) {
    spacesApi.getSpace(demoSpace.spaceId)
        .then(spaceDetails => {
            console.log('🔍 Space Configuration:');
            console.log(`   Name: ${spaceDetails.name}`);
            console.log(`   Owner ID: ${spaceDetails.ownerId}`);
            console.log(`   Public Read: ${spaceDetails.publicRead}`);
            console.log(`   Created: ${new Date(spaceDetails.createdAt).toISOString()}`);
            console.log(`   Labels: ${JSON.stringify(spaceDetails.labels)}`);
            
            console.log('\n🤖 Associated Embedders:');
            if (spaceDetails.spaceEmbedders && spaceDetails.spaceEmbedders.length > 0) {
                spaceDetails.spaceEmbedders.forEach(embedderAssoc => {
                    console.log(`   Embedder ID: ${embedderAssoc.embedderId}`);
                    console.log(`   Retrieval Weight: ${embedderAssoc.defaultRetrievalWeight}`);
                });
            } else {
                console.log('   No embedders configured');
            }
        })
        .catch(error => {
            console.log('❌ Error getting space details:', error.message);
        });
} else {
    console.log('⚠️  No space available for the demo');
}

## Adding Documents to Memory

### The Document Processing Pipeline

When you add a document to GoodMem, it goes through several automated steps:

```
1. Ingestion → 2. Chunking → 3. Embedding → 4. Indexing → 5. Ready for Search
```

**What happens**:
1. **Ingestion**: Document content and metadata are stored
2. **Chunking**: Text is split according to your configuration (256 chars, 25 overlap)
3. **Embedding**: Each chunk is converted to a vector by your embedder
4. **Indexing**: Vectors are indexed for fast similarity search
5. **Status**: Document marked as `COMPLETED` and ready for retrieval

### Single vs. Batch Operations

**Single memory creation** (`CreateMemory`):
- ✅ Good for: Real-time ingestion, single documents
- ✅ Synchronous processing with immediate status
- ⚠️ Higher overhead for bulk operations

**Batch memory creation** (`BatchCreateMemory`):
- ✅ Good for: Bulk imports, initial setup, periodic updates
- ✅ Lower overhead, efficient for multiple documents
- ✅ Async processing - check status via `ListMemories`
- ⚠️ Takes longer to get individual status feedback

### Metadata Best Practices

Rich metadata helps with:
- **Filtering**: Retrieve specific document types
- **Source attribution**: Show users where information came from
- **Organization**: Group and manage related documents
- **Debugging**: Track ingestion methods and dates

### What We'll Do

1. Load sample documents from local files
2. Create one document using single memory creation (to demo the API)
3. Create remaining documents using batch operation (more efficient)
4. Monitor processing status until all documents are ready

We'll use sample company documents that represent common business use cases:

In [ ]:
// Load our sample documents
async function loadSampleDocuments() {
    const documents = [];
    const sampleDir = 'sample_documents';
    
    const docFiles = {
        'company_handbook.txt': 'Employee handbook with policies and procedures',
        'technical_documentation.txt': 'API documentation and technical guides',
        'product_faq.txt': 'Frequently asked questions about products',
        'security_policy.txt': 'Information security policies and procedures'
    };
    
    for (const [filename, description] of Object.entries(docFiles)) {
        const filepath = path.join(sampleDir, filename);
        
        try {
            const content = await fs.readFile(filepath, 'utf-8');
            documents.push({ filename, description, content });
            console.log(`📄 Loaded: ${filename} (${content.length.toLocaleString()} characters)`);
        } catch (error) {
            console.log(`⚠️  File not found: ${filepath}`);
        }
    }
    
    return documents;
}

// Load the documents
let sampleDocs = [];
loadSampleDocuments()
    .then(docs => {
        sampleDocs = docs;
        console.log(`\n📚 Total documents loaded: ${docs.length}`);
    })
    .catch(error => {
        console.log('❌ Error loading documents:', error.message);
    });

In [ ]:
// Create the first memory individually to demonstrate single memory creation
function createSingleMemory(spaceId, document) {
    const memoryRequest = {
        spaceId: spaceId,
        originalContent: document.content,
        contentType: 'text/plain',
        chunkingConfig: DEMO_CHUNKING_CONFIG,
        metadata: {
            filename: document.filename,
            description: document.description,
            source: 'sample_documents',
            document_type: document.filename.split('_')[0],
            ingestion_method: 'single'
        }
    };
    
    return memoriesApi.createMemory(memoryRequest)
        .then(memory => {
            console.log(`✅ Created single memory: ${document.filename}`);
            console.log(`   Memory ID: ${memory.memoryId}`);
            console.log(`   Status: ${memory.processingStatus}`);
            console.log(`   Content Length: ${document.content.length} characters`);
            console.log();
            return memory;
        })
        .catch(error => {
            console.log(`❌ Error creating memory for ${document.filename}:`, error.message);
            return null;
        });
}

let singleMemory = null;
if (demoSpace && sampleDocs.length > 0) {
    const firstDoc = sampleDocs[0];
    console.log('📝 Creating first document using CreateMemory API:');
    console.log(`   Document: ${firstDoc.filename}`);
    console.log('   Method: Individual memory creation');
    console.log();
    
    createSingleMemory(demoSpace.spaceId, firstDoc)
        .then(memory => {
            singleMemory = memory;
            if (memory) {
                console.log('🎯 Single memory creation completed successfully!');
            } else {
                console.log('⚠️  Single memory creation failed');
            }
        });
} else {
    console.log('⚠️  Cannot create memory: missing space or documents');
}

In [ ]:
// Demonstrate retrieving a memory by ID using getMemory
if (singleMemory) {
    console.log('📖 Retrieving memory details using getMemory API:');
    console.log(`   Memory ID: ${singleMemory.memoryId}`);
    console.log();
    
    // Retrieve the memory without content
    memoriesApi.getMemory(singleMemory.memoryId, { includeContent: false })
        .then(retrievedMemory => {
            console.log('✅ Successfully retrieved memory:');
            console.log(`   Memory ID: ${retrievedMemory.memoryId}`);
            console.log(`   Space ID: ${retrievedMemory.spaceId}`);
            console.log(`   Status: ${retrievedMemory.processingStatus}`);
            console.log(`   Content Type: ${retrievedMemory.contentType}`);
            console.log(`   Created At: ${new Date(retrievedMemory.createdAt).toISOString()}`);
            console.log(`   Updated At: ${new Date(retrievedMemory.updatedAt).toISOString()}`);
            
            if (retrievedMemory.metadata) {
                console.log('\n   📋 Metadata:');
                for (const [key, value] of Object.entries(retrievedMemory.metadata)) {
                    console.log(`      ${key}: ${value}`);
                }
            }
            
            // Now retrieve with content included
            console.log('\n📖 Retrieving memory with content:');
            return memoriesApi.getMemory(singleMemory.memoryId, { includeContent: true });
        })
        .then(retrievedWithContent => {
            if (retrievedWithContent.originalContent) {
                // Decode the base64 encoded content
                const decodedContent = Buffer.from(retrievedWithContent.originalContent, 'base64').toString('utf-8');
                
                console.log('✅ Content retrieved and decoded:');
                console.log(`   Content length: ${decodedContent.length} characters`);
                const preview = decodedContent.length > 200 ? decodedContent.substring(0, 200) + '...' : decodedContent;
                console.log(`   First 200 chars: ${preview}`);
            } else {
                console.log('⚠️  No content available');
            }
        })
        .catch(error => {
            console.log('❌ Error retrieving memory:', error.message);
            if (error.status) {
                console.log('   Status code:', error.status);
            }
        });
} else {
    console.log('⚠️  No memory available to retrieve');
}

In [ ]:
// Create the remaining documents using batch memory creation
function createBatchMemories(spaceId, documents) {
    const memoryRequests = documents.map(doc => ({
        spaceId: spaceId,
        originalContent: doc.content,
        contentType: 'text/plain',
        chunkingConfig: DEMO_CHUNKING_CONFIG,
        metadata: {
            filename: doc.filename,
            description: doc.description,
            source: 'sample_documents',
            document_type: doc.filename.split('_')[0],
            ingestion_method: 'batch'
        }
    }));
    
    const batchRequest = {
        requests: memoryRequests
    };
    
    console.log(`📦 Creating ${memoryRequests.length} memories using BatchCreateMemory API:`);
    
    return memoriesApi.batchCreateMemory(batchRequest)
        .then(() => {
            console.log('✅ Batch creation request submitted successfully');
        })
        .catch(error => {
            console.log('❌ Error during batch creation:', error.message);
            if (error.status) {
                console.log('   Response code:', error.status);
            }
        });
}

if (demoSpace && sampleDocs.length > 1) {
    const remainingDocs = sampleDocs.slice(1);
    createBatchMemories(demoSpace.spaceId, remainingDocs)
        .then(() => {
            console.log('\n📋 Total Memory Creation Summary:');
            console.log('   📄 Single CreateMemory: 1 document');
            console.log(`   📦 Batch CreateMemory: ${remainingDocs.length} documents submitted`);
            console.log('   ⏳ Check processing status in the next cell');
        });
} else {
    console.log('⚠️  Cannot create batch memories: insufficient documents or missing space');
}

In [ ]:
// List all memories in our space to verify they're ready
if (demoSpace) {
    memoriesApi.listMemories(demoSpace.spaceId)
        .then(memoriesResponse => {
            const memories = memoriesResponse.memories || [];
            
            console.log(`📚 Memories in space '${demoSpace.name}':`);
            console.log(`   Total memories: ${memories.length}`);
            console.log();
            
            memories.forEach((memory, i) => {
                const metadata = memory.metadata || {};
                const filename = metadata.filename || 'Unknown';
                const description = metadata.description || 'No description';
                
                console.log(`   ${i + 1}. ${filename}`);
                console.log(`      Status: ${memory.processingStatus}`);
                console.log(`      Description: ${description}`);
                console.log(`      Created: ${new Date(memory.createdAt).toISOString()}`);
                console.log();
            });
        })
        .catch(error => {
            console.log('❌ Error listing memories:', error.message);
        });
}

In [ ]:
// Monitor processing status for all created memories
async function waitForProcessingCompletion(spaceId, maxWaitSeconds = 120) {
    console.log('⏳ Waiting for document processing to complete...');
    console.log('   💡 Note: Batch memories are processed asynchronously, so we check by listing all memories in the space');
    console.log();
    
    const startTime = Date.now();
    const maxWaitMs = maxWaitSeconds * 1000;
    
    while (Date.now() - startTime < maxWaitMs) {
        try {
            const memoriesResponse = await memoriesApi.listMemories(spaceId);
            const memories = memoriesResponse.memories || [];
            
            // Check processing status
            const statusCounts = {};
            memories.forEach(memory => {
                const status = memory.processingStatus;
                statusCounts[status] = (statusCounts[status] || 0) + 1;
            });
            
            console.log(`📊 Processing status: ${JSON.stringify(statusCounts)} (Total: ${memories.length} memories)`);
            
            // Check if all are completed
            const allCompleted = memories.every(memory => memory.processingStatus === 'COMPLETED');
            if (allCompleted) {
                console.log('✅ All documents processed successfully!');
                return true;
            }
            
            // Check for any failures
            const failedCount = statusCounts['FAILED'] || 0;
            if (failedCount > 0) {
                console.log(`❌ ${failedCount} memories failed processing`);
                return false;
            }
            
            await new Promise(resolve => setTimeout(resolve, 5000)); // Wait 5 seconds
        } catch (error) {
            console.log('❌ Error checking processing status:', error.message);
            return false;
        }
    }
    
    console.log(`⏰ Timeout waiting for processing (waited ${maxWaitSeconds}s)`);
    return false;
}

if (demoSpace) {
    waitForProcessingCompletion(demoSpace.spaceId)
        .then(processingComplete => {
            if (processingComplete) {
                console.log('🎉 Ready for semantic search and retrieval!');
                console.log('📈 Batch API benefit: Multiple documents submitted in a single API call');
                console.log('🔧 Consistent chunking: All memories use DEMO_CHUNKING_CONFIG');
            } else {
                console.log('⚠️  Some documents may still be processing. You can continue with the tutorial.');
            }
        });
} else {
    console.log('⚠️  Skipping processing check - no space available');
}

## Semantic Search & Retrieval

### Why Semantic Search?

**Traditional keyword search**:
- Matches exact words or simple variations
- Misses conceptually similar content with different wording
- Example: "vacation days" won't match "time off policy"

**Semantic search**:
- Understands meaning and context
- Finds conceptually similar content regardless of exact wording
- Example: "vacation days" successfully matches "time off policy"

### How It Works

```
Query: "vacation policy" 
   ↓ (embed with same embedder)
Query Vector: [0.23, -0.45, ...]
   ↓ (compare to all chunk vectors)
Most Similar Chunks: (by cosine similarity)
   1. "TIME OFF POLICY..." (score: -0.604)
   2. "Vacation requests..." (score: -0.544)
   3. "WORK HOURS..." (score: -0.458)
```

### Understanding Relevance Scores

GoodMem uses **cosine distance** (negative cosine similarity):
- **Lower values = more relevant** (e.g., -0.6 is better than -0.4)
- **Range**: Typically -1.0 (most similar) to 0.0 (unrelated)
- **Good threshold**: Results under -0.3 are usually relevant
- **Context matters**: Exact scores vary by embedder and content

### Streaming API Benefits

GoodMem's streaming API:
- **Real-time results**: Process chunks as they arrive
- **Low latency**: Start showing results immediately
- **Memory efficient**: No need to buffer entire result set
- **Progressive UI**: Update interface as more results come in

### What We'll Do

1. Implement a semantic search function using GoodMem's streaming API
2. Process different event types (chunks, memories, metadata)
3. Display results with relevance scores
4. Test with various queries to see semantic matching in action

Now comes the exciting part! Let's perform semantic search using GoodMem's streaming API. This will:

- **Find relevant chunks** based on semantic similarity
- **Stream results** in real-time
- **Include relevance scores** for ranking
- **Return structured data** for easy processing

In [ ]:
// Perform semantic search using GoodMem's streaming API
async function semanticSearchStreaming(query, spaceId, maxResults = 5) {
    console.log(`🔍 Streaming search for: '${query}'`);
    console.log(`📁 Space ID: ${spaceId}`);
    console.log(`📊 Max results: ${maxResults}`);
    console.log('-'.repeat(50));
    
    try {
        const controller = new AbortController();
        const timeoutId = setTimeout(() => {
            console.log('⏰ Streaming timeout after 30 seconds');
            controller.abort();
        }, 30000);
        
        // Create request object with proper structure
        const request = {
            message: query,
            spaceIds: [spaceId],
            requestedSize: maxResults,
            fetchMemory: true,
            fetchMemoryContent: false,
            format: GoodMemClient.StreamingClient.StreamingFormat.NDJSON
        };
        
        const stream = await streamingClient.retrieveMemoryStream(
            controller.signal,
            request
        );
        
        const retrievedChunks = [];
        let eventCount = 0;
        
        for await (const event of stream) {
            eventCount++;
            
            if (event.retrievedItem && event.retrievedItem.chunk) {
                const chunkRef = event.retrievedItem.chunk;
                const chunkData = chunkRef.chunk || {};
                const chunkText = chunkData.chunkText || '';
                
                retrievedChunks.push({
                    chunkText: chunkText,
                    relevanceScore: chunkRef.relevanceScore,
                    memoryIndex: chunkRef.memoryIndex,
                    resultSetId: chunkRef.resultSetId,
                    chunkSequence: chunkData.chunkSequenceNumber || 0
                });
                
                console.log(`📄 Chunk ${retrievedChunks.length}:`);
                console.log(`   Relevance: ${chunkRef.relevanceScore.toFixed(3)}`);
                const preview = chunkText.length > 100 ? chunkText.substring(0, 100) + '...' : chunkText;
                console.log(`   Text: ${preview}`);
                console.log();
            } else if (event.resultSetBoundary) {
                console.log(`🔄 ${event.resultSetBoundary.kind}: ${event.resultSetBoundary.stageName}`);
            }
        }
        
        clearTimeout(timeoutId);
        
        console.log(`✅ Streaming search completed: ${retrievedChunks.length} chunks found, ${eventCount} events processed`);
        return retrievedChunks;
    } catch (error) {
        if (error.name === 'AbortError' || error.message.includes('aborted')) {
            console.log('Stream completed or cancelled');
        } else {
            console.log('❌ Streaming error:', error.message);
        }
        return [];
    }
}

// Test semantic search with a sample query
if (demoSpace && streamingClient) {
    const sampleQuery = 'What is the vacation policy for employees?';
    semanticSearchStreaming(sampleQuery, demoSpace.spaceId)
        .then(searchResults => {
            console.log(`\n✅ Search completed with ${searchResults.length} results`);
        });
} else if (!streamingClient) {
    console.log('⚠️  StreamingClient not available for search');
} else {
    console.log('⚠️  No space available for search');
}

In [ ]:
// Let's try a few different queries to see how streaming semantic search works
async function testMultipleStreamingQueries(spaceId) {
    const testQueries = [
        'How do I reset my password?',
        'What are the security requirements for remote work?',
        'API authentication and rate limits',
        'Employee benefits and health insurance',
        'How much does the software cost?'
    ];
    
    for (let i = 0; i < testQueries.length; i++) {
        const query = testQueries[i];
        console.log(`\n🔍 Test Query ${i + 1}: ${query}`);
        console.log('='.repeat(60));
        
        await semanticSearchStreaming(query, spaceId, 3);
        
        console.log('\n' + '-'.repeat(60));
    }
}

if (demoSpace && streamingClient) {
    testMultipleStreamingQueries(demoSpace.spaceId)
        .then(() => console.log('\n✅ All queries completed'));
} else if (!streamingClient) {
    console.log('⚠️  StreamingClient not available for testing multiple queries');
} else {
    console.log('⚠️  No space available for testing multiple streaming queries');
}

## Advanced Features

Congratulations! 🎉 You've successfully built a semantic search system using GoodMem. Here's what you've accomplished:

### ✅ What You Built
- **Document ingestion pipeline** with automatic chunking and embedding
- **Semantic search system** with relevance scoring
- **Simple Q&A system** using GoodMem's vector capabilities

### 🚀 Next Steps for Advanced Implementation

#### Reranking
Improve search quality by adding a reranking stage. **Rerankers** are specialized models that re-score search results to improve relevance:

- **Two-stage retrieval**: Fast initial retrieval with embeddings, then precise reranking
- **Better relevance**: Rerankers use cross-attention to understand query-document relationships
- **Reduced costs**: Rerank only top-K results instead of entire corpus
- **Voyage AI reranker**: Industry-leading reranking model with state-of-the-art performance

The combination of fast embedding-based retrieval followed by accurate reranking provides the best balance of speed and quality for production RAG systems.

## Configuring a Reranker

To further improve search quality, we can add a **reranker** to our RAG pipeline. While embedders provide fast semantic search, rerankers use more sophisticated models to re-score the top results for better accuracy.

### Why Use Reranking?

1. **Higher Accuracy**: Rerankers use cross-encoder architectures that directly compare queries and documents
2. **Two-Stage Pipeline**: Fast retrieval with embeddings + precise reranking = optimal performance
3. **Cost Effective**: Only rerank top-K results (e.g., top 20) rather than entire corpus

### Voyage AI Reranker

We'll use Voyage AI's `rerank-2.5` model, which provides:
- **State-of-the-art performance** on reranking benchmarks
- **Fast inference** optimized for production use
- **Simple API** that integrates seamlessly with GoodMem

**Note**: You'll need a Voyage AI API key set in your environment variable `VOYAGE_API_KEY`.

In [ ]:
// JavaScript SDK reranker creation - Coming soon


## Registering an LLM

The final component in our RAG pipeline is the **LLM (Large Language Model)** - the generation component that creates natural language responses using the retrieved and reranked context.

### Role of LLMs in RAG

After retrieving and reranking relevant chunks, the LLM:
1. **Receives the query** and retrieved context
2. **Generates a response** that synthesizes information from multiple sources
3. **Maintains coherence** while staying grounded in the retrieved facts

### OpenAI GPT-4o-mini

We'll use OpenAI's `gpt-4o-mini` model, which provides:
- **Fast inference** with low latency for real-time applications
- **Cost-effective** pricing compared to larger models
- **High quality** responses suitable for most RAG use cases
- **Function calling** support for advanced workflows

**Note**: This uses the same `OPENAI_API_KEY` environment variable as the embedder.

In [ ]:
// JavaScript SDK LLM registration - Coming soon


## Enhanced RAG with Reranking and LLM Generation

Now that we have all the components configured (embedder, reranker, and LLM), let's use the complete RAG pipeline! This demonstrates the full power of GoodMem:

1. **Retrieval**: Fast semantic search finds relevant chunks
2. **Reranking**: Voyage AI reranker re-scores results for better relevance  
3. **Generation**: OpenAI GPT-4o-mini generates a coherent response using the reranked context

This provides significantly better answer quality compared to simple retrieval alone.

In [ ]:
// JavaScript SDK complete RAG pipeline - Coming soon


## 🎉 Congratulations! What You Built

You've successfully built a complete **Retrieval-Augmented Generation (RAG) system** using GoodMem! Let's recap what you accomplished.

### Components You Configured

| Component | Purpose | Function |
|-----------|---------|----------|
| **Embedder** | Convert text to vectors | Transform documents into semantic embeddings |
| **Space** | Organize and store documents | Logical container with chunking configuration |
| **Memories** | Store searchable content | Documents chunked and indexed for retrieval |
| **Reranker** | Improve search precision | Re-score results for better relevance |
| **LLM** | Generate natural language | Create coherent answers from retrieved context |

### The Complete RAG Pipeline

```
📄 Documents
   ↓ Chunking (256 chars, 25 overlap)
   ↓ Embedding (convert to vectors)
🗄️  Vector Storage (GoodMem Space)
   ↓ 
🔍 User Query
   ↓ Semantic Search (retrieve top-K)
   ↓ Reranking (re-score for precision)
   ↓ Context Selection (most relevant chunks)
🤖 LLM Generation (synthesize answer)
   ↓
✨ Natural Language Answer
```

### Key Concepts You Learned

1. **Embedders**: Transform text into semantic vectors for similarity search
2. **Spaces**: Logical containers for organizing and searching documents
3. **Chunking**: Breaking documents into optimal sizes for retrieval
4. **Semantic Search**: Finding conceptually similar content, not just keyword matches
5. **Reranking**: Two-stage retrieval for better precision
6. **Streaming API**: Real-time, memory-efficient result processing
7. **RAG Architecture**: Combining retrieval and generation for accurate, grounded responses

### Performance Improvements

**Basic search** (retrieval only):
- Fast retrieval using vector similarity
- Good recall, but may include less relevant results

**Enhanced RAG** (with reranker + LLM):
- Reranker improves precision significantly
- LLM synthesizes information from multiple chunks
- Better user experience with natural language answers
- Grounded in actual document content (no hallucinations)

### Next Steps & Advanced Topics

**Enhance Your RAG System**:
- **Multiple embedders**: Combine different embedders for better coverage
- **Custom chunking**: Tune chunk size/overlap for your content type
- **Metadata filtering**: Add filters to narrow search by document type, date, etc.
- **Hybrid search**: Combine semantic and keyword search
- **Context augmentation**: Include surrounding chunks for better LLM context

**Production Deployment**:
- **Monitoring**: Track query latency, relevance scores, user feedback
- **Scaling**: Horizontal scaling for high-traffic applications
- **Cost optimization**: Balance quality vs. API costs
- **Caching**: Cache frequent queries for faster responses
- **Error handling**: Robust exception handling and retry logic

**Advanced Features**:
- **Multi-space search**: Query across multiple knowledge bases
- **Query expansion**: Transform queries for better retrieval
- **Result aggregation**: Combine and deduplicate results
- **Streaming generation**: Progressive LLM responses for real-time UX
- **Fine-tuning**: Customize models for your specific domain

### Resources

- **Documentation**: [https://docs.goodmem.ai](https://docs.goodmem.ai)
- **Community**: Join discussions and share your implementations
- **Examples**: Explore more advanced use cases and patterns

---

**Great job!** You now have a solid foundation for building production RAG systems with GoodMem. 🚀
